In [2]:
import pandas as pd
import numpy as np
import pickle
import math
import warnings
# !pip install yfinance
import yfinance as yf
"""Evaluate trading strategy performance over time interval."""
from typing import List, Tuple, Dict
from datetime import datetime
#import datetime

warnings.filterwarnings("ignore")



In [35]:
(0.077511+1)**(365.25/90)-1

0.35387358933569146

In [36]:
pd.read_table("8k_data_labels.tsv",)


,Unnamed: 0.1,Unnamed: 0,tikr,FORM_TYPEs,ubmission,text,Date,label,start price,annualized return,sp annualized return,percent return,1_day_after_moving_avg,7_day_after_moving_avg,30_day_after_moving_avg,90_day_after_moving_avg
0,0,0,aapl,8-K-MULTI,0000320193-17-000006,8-K 1 a8-kq32017712017.htm 8-K Document UNITE...,20170801,1,39.285000,0.353871,0.287963,0.077511,37.935000,39.441786,40.210357,42.505357
1,1,1,aapl,8-K-None,0000320193-17-000006,EX-99.1 2 a8-kexhibit991q32017712017.htm EXHI...,20170801,1,39.285000,0.353871,0.287963,0.077511,37.935000,39.441786,40.210357,42.505357
2,2,2,aapl,8-K-None,0000320193-17-000006,EX-99.2 3 a8-kexhibit992q32017712017.htm EXHI...,20170801,1,39.285000,0.353871,0.287963,0.077511,37.935000,39.441786,40.210357,42.505357
3,3,3,aapl,8-K-MULTI,0000320193-17-000067,8-K 1 a8-kq420179302017.htm 8-K Document UNIT...,20171102,1,43.125000,0.152771,0.274702,0.035652,41.561429,43.654286,42.911429,44.668928
4,4,4,aapl,8-K-None,0000320193-17-000067,EX-99.1 2 a8-kexhibit991q420179302017.htm EXH...,20171102,1,43.125000,0.152771,0.274702,0.035652,41.561429,43.654286,42.911429,44.668928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,563,563,aapl,8-K-None,0001628280-17-000663,EX-99.1 2 a8-kexhibit991q12017123120.htm EXHI...,20170131,1,32.187500,0.808090,0.299963,0.157126,30.623928,32.594285,34.821072,38.441071
530,564,564,aapl,8-K-None,0001628280-17-000663,EX-99.2 3 a8-kexhibit992q12017123120.htm EXHI...,20170131,1,32.187500,0.808090,0.299963,0.157126,30.623928,32.594285,34.821072,38.441071
531,565,565,aapl,8-K-MULTI,0001628280-17-004663,8-K 1 a8-kq22017412017.htm 8-K Document UNITE...,20170502,1,36.764999,0.359821,0.130535,0.078675,36.314285,37.741428,37.534285,40.545715
532,566,566,aapl,8-K-None,0001628280-17-004663,EX-99.1 2 a8-kexhibit991q22017412017.htm EXHI...,20170502,1,36.764999,0.359821,0.130535,0.078675,36.314285,37.741428,37.534285,40.545715


In [22]:
#train_df = pd.read_csv('8k_data_labels.csv')
outlier = pd.read_csv('outlier.csv')
outlier

,start_date,days_period,start_price,n_day_price,n_day_return
0,2018-02-01,7,40.125000,40.677502,1.041269
1,2018-02-01,30,40.125000,44.505001,2.530199
2,2018-02-01,90,40.125000,48.070000,1.081676
3,2018-02-01,7,40.125000,40.677502,1.041269
4,2018-02-01,30,40.125000,44.505001,2.530199
...,...,...,...,...,...
676,2017-01-31,7,32.187500,33.105000,3.334204
677,2017-01-31,30,32.187500,35.115002,1.885900
678,2017-05-02,7,36.764999,38.487499,9.903795
679,2017-05-02,7,36.764999,38.487499,9.903795


In [28]:
for i in range(len(outlier)):
    row = outlier.iloc[i]
    check_return = (row.loc["n_day_price"]/row.loc["start_price"]) ** (365.25/row.loc["days_period"]) - 1
    if(round(check_return, 4) != round(row.loc["n_day_return"],4)):
        print(
            row.loc["start_date"],
            row.loc["days_period"],
            row.loc["start_price"], 
            row.loc["n_day_price"],
            row.loc["n_day_price"]/row.loc["start_price"],
            round(check_return, 4),
            round(row.loc["n_day_return"],4))

1997-07-28 7 0.1473210006952285 0.234933003783226 1.5947013845585094 37628772569.2783 37628772569.2772


In [92]:
def load_historical_data(filename):
    with open(filename, 'rb') as handle:
        tikr_dict = pickle.load(handle)
        return tikr_dict
TIKRS_dat = load_historical_data('TIKR_DATA.pickle')

In [175]:
class StockSimulation:
    def __init__(self, cash=1000000, tikrs = ['aapl','msft'], historical_data=TIKRS_dat):
        self.cash = cash
        self.tikrs = tikrs
        self.portfolio = {}
        for tikr in tikrs:
            self.portfolio[tikr] = 0
        self.active_log = []
        self.transaction_log = []
        self.transaction_cost = 0.01
        self.historical_data = TIKRS_dat
        
    def get_price(self, tikr, date):
        start = date
        if type(start) is str:
            start = datetime.strptime(start, '%Y%m%d')
          
        date = start.strftime('%Y-%m-%d')

        company_df = self.historical_data[tikr]
        return company_df[company_df['Date'] > date].iloc[0]['Open']
#         return yf.download(
#             tikr, 
#             start=start,
#             interval= '1d', 
#             progress= False, ).loc[date]['Adj Close']



    def buy(self, tikr, date, allocated_money):
        # check if there is enough cash to buy
        if self.cash < allocated_money:
            print("Not enough cash to buy")
            return

        # calculate number of shares to buy
        price = self.get_price(tikr, date)
        shares = allocated_money / price

        # update portfolio and cash balance
        if tikr in self.active_log:
            self.portfolio[tikr] += shares
        else:
            self.portfolio[tikr] = shares
            self.active_log.append(tikr)
        # minus the cost of stock
        stock_cost = shares * price
        # minus trasaction cost
        #self.cash -= stock_cost * (1 + self.transaction_cost)
        self.cash -= stock_cost 


        # log transaction
        self.transaction_log.append({
            "type": "buy",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": 0,
            "balance": self.active_balance(date)
        })

    def sell(self, tikr, date, allocated_money):
        # check if there are enough shares to sell
        if tikr not in self.portfolio:
            print("No shares of {} in portfolio".format(tikr))
            return


        # calculate number of shares to sell
        price = self.get_price(tikr, date)
        shares = allocated_money / price


        if self.portfolio[tikr] < shares:
            print("Not enough shares of {} to sell".format(tikr))
            return

        # update portfolio and cash balance
        self.portfolio[tikr] -= shares
        if self.portfolio[tikr] == 0:
            self.active_log.remove(tikr)

        # plus the earning
        stock_cost = shares * price
        self.cash += stock_cost
        # minus trasaction cost
        self.cash -= stock_cost *  self.transaction_cost


        # log transaction
        self.transaction_log.append({
            "type": "sell",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": stock_cost * self.transaction_cost,
            "balance": self.active_balance(date)
        })

    #TODO
    def get_next_trading_date(self,tikr, date):
        return None

    def rebalance(self,percentage,date):
        buy = []
        
        
        true_balance = self.active_balance(date) * 0.99
        for tikr, percent in zip(self.tikrs, percentage):
            price = self.get_price(tikr, date)
            expected_value = true_balance * percent
            current_value = self.portfolio[tikr] * price
            allocated_money = expected_value - current_value

            if allocated_money < 0:
                self.sell(tikr, date, -1 * allocated_money)
                #total_transaction += self.transaction_log[-1]['transaction_cost']
            else:
                buy += [(tikr, allocated_money )]
        
       # avg_transaction = total_transaction/len(buy)
        for tikr, allocated_money in buy:
            self.buy(tikr, date, allocated_money )#- avg_transaction)




    def active_balance(self, date):
        balance = self.cash
        for tikr in self.active_log:
            price = self.get_price(tikr, date)
            balance += self.portfolio[tikr] * price
        return balance
    
    def print_portfolio(self, date):
        balance = self.active_balance(date)
        for tikr in self.tikrs:
            price = self.get_price(tikr, date)
            tikr_holding = self.portfolio[tikr] * price
            print(tikr, tikr_holding/balance )
            

    def transaction_summary(self):
        for txn in self.transaction_log:
            print("{} {} {} shares of {} at ${:.2f} for ${:.2f} (transaction cost: ${:.2f}), balance: ${:.2f}".format(
                txn['date'], txn['type'], txn['shares'], txn['tikr'], txn['price'], txn['amount'], txn['transaction_cost'], txn['balance']))


In [176]:
a =[[1,2,3],[3,2,1]]
np.argmax(a, axis=1)

array([2, 0])

In [177]:



""" All trading strategies should have standardized input to work in automated
    downstream testing."""

"""
def trading_strategy(
        dates,
        predictions,
        company_list: List[str]
            ) -> Dict[str, List[float]]:
    
    strategy = dict()
    predictions = np.argmax(predictions, axis=1)
    for date, tikr, pred in zip(dates, tikrs, predictions):
        if pred == 0: #underperforms
            
            #sell
            
            
        
    
    return strategy
"""

def get_strategy_annual_return(
        strategy: Dict[str, List[float]],
        company_list: List[str],
        end_date,
        starting_balance: int = 1e7,
        start_date: datetime = '20000101') -> float:
    """
    Calculate annual return over time period.
    
    Parameters
    ----------
    strategy: Dict[datetime, List[float]]
        A dictionary of portfolio rebalance dates associated with portfolio
        allocation percentages. The portfolio allocation percentages share the
        same indexing as company_list.
    company_list: List[str]
        The companies invested into by the trading strategy percent allocation.
    start_date: datetime, str
        Accepted format "year_month_day", or datetime object. The first day of
        trading.
    end_date: datetime, str
        Accepted format "year_month_day", or datetime object. The final day of
        trading used to determine net worth and annualized return.
    starting_balance: int
        The initial amount of money invested.
    Returns
    -------
    Annualized Return: float
        The annualized return rate, where 0% return indicates 1.00.
    """

    if type(start_date) is str:
        start_date = datetime.strptime(start_date, "%Y%m%d")
    if type(end_date) is str:
        end_date = datetime.strptime(end_date, "%Y%m%d")

    s = StockSimulation(cash = starting_balance, tikrs = company_list)
    for date, portfolio_allocation in strategy.items():
        try:
            s.rebalance(percentage= portfolio_allocation, date = date)
            print(date, " balance:",s.active_balance(date))
        except:
            print('rebalance')




    # At each date, rebalance current networth to be distributed
    # percentage-wise between companies in portfolio_allocations

    return s.active_balance(end_date)/starting_balance

In [178]:
s = StockSimulation()
s.get_price('msft', '20000110')
s.buy('msft', '20000110', 10000)
s.sell('msft', '20000110', 10000)
s.active_balance('20000110')


999900.0

In [179]:
s = StockSimulation(cash =1000)
s.rebalance([0.5,0.5],'20001210')
s.print_portfolio( '20001210')
s.rebalance([0.3,0.7], "20011210")
s.print_portfolio( "20011210")
s.rebalance([0,0],'20100112')

aapl 0.495
msft 0.495
aapl 0.29774248620736027
msft 0.694732467817174


In [173]:
s.transaction_summary()

20001210 buy 1825.1875346835704 shares of aapl at $0.27 for $495.00 (transaction cost: $0.00), balance: $1000.00
20001210 buy 17.83783783783784 shares of msft at $27.75 for $495.00 (transaction cost: $0.00), balance: $1000.00


8132.000126617264

In [161]:
company_list = ['aapl', 'msft']
end_date = "20100112"
start_date = '20001210'
starting_balance = 1000
strategy = {"20001210": [0.5,0.5], "20011210": [0.3,0.7],"20100112": [0,0] }

In [162]:
get_strategy_annual_return(strategy,company_list,end_date,starting_balance, start_date)

20001210  balance: 1000.0
20011210  balance: 1348.9560177068931
20100112  balance: 8132.000126617264


8.132000126617264